# Evaluating trained model with large dataset

## Preparation

### Importing libraries

In [1]:
import os
import sys
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import load_model

%matplotlib inline

### Image conversion functions

In [2]:
def png_to_jpeg(png_file, jpeg_file):
    """
    Convert PNG images to JPEG format
    :param png_file: full path of .png file
    :param jpeg_file: full path of .jpeg file
    """
    im = PIL.Image.open(png_file)
    rgb_im = im.convert('RGB')
    rgb_im.save(jpeg_file, 'JPEG')
    
def convert_images(folder):
    jpeg_dir = folder + '/jpeg'

    if not os.path.exists(jpeg_dir):
        os.makedirs(jpeg_dir)
    image_paths = list(paths.list_images(folder))

    for image_path in image_paths:
        # if image_path.split('.')[-1].lower() == 'png':
        jpeg_image_path = image_path.replace(
            os.path.dirname(image_path),
            jpeg_dir
        )
        jpeg_image_path = jpeg_image_path.replace('.png', '.jpg')
        # convert images to jpeg if they don't already exist
        if not os.path.isfile(jpeg_image_path):
            png_to_jpeg(image_path, jpeg_image_path)

### Converting images to `jpeg`

In [3]:
light_colors = ['Green', 'Red', 'Yellow', 'Unknown']
folder = 'data/test_images/{}'

for color in light_colors:
    convert_images(folder.format(color))

### Loading dataset

In [4]:
num_classes = 4
WIDTH = 100
HEIGHT = 100
image_folders = 'data/test_images/{}/jpeg/'

data = []
labels = []

for color in light_colors:
    imagePaths = list(paths.list_images(image_folders.format(color)))

    for imagePath in imagePaths:
        image = cv2.imread(imagePath, -1)
        image = cv2.resize(image, (HEIGHT, WIDTH))
        image = np.reshape(image, (HEIGHT, WIDTH, 3))

        data.append(image)
        labels.append(color)

data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

### Reformat labels

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

## Evaluating model

### Loading model

In [14]:
model = load_model('saved_model/trained_cnn')

### Evaluating dataset

In [15]:
test_loss, test_acc = model.evaluate(data,  labels, verbose=2)
print(test_acc, test_loss)

141/141 - 20s - loss: 3.6550 - accuracy: 0.7273
0.7272727489471436 3.6550066471099854
